In [ ]:
!pip install unsloth

**Load Model**

In [ ]:
!pip install --upgrade transformers>=4.49.0

In [ ]:
from unsloth import FastModel
import torch

MODEL = "unsloth/gemma-3-4b-it"

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

**Defining LORA Config**

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r = 4,
    lora_alpha = 4,
    lora_dropout = 0,
    bias = "none",
)

RuntimeError: Unsloth: You already added LoRA adapters to your model!

**Dataset**

In [ ]:
from datasets import load_dataset
from unsloth import to_sharegpt
from unsloth.chat_templates import standardize_data_formats

In [ ]:
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

dataset = standardize_data_formats(dataset)



README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }

dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

**Trainer**

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(model = model,
                     tokenizer = tokenizer,
                     train_dataset = dataset,
                     args = SFTConfig(
                         per_device_train_batch_size = 2,
                         gradient_accumulation_steps = 4,
                         max_steps = 50,
                         learning_rate = 2e-4,
                         optim = "adamw_8bit",
                         weight_decay = 0.01,
                         lr_scheduler_type = "linear", #it avoids abrupt learning rate reductions, leading to better final performance.
                         warmup_ratio = 0.03,
                     ))

Unsloth: Switching to float32 training since model cannot work with float16
Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vatsajoshi2 (vatsajoshi2-navrachana-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.852000
2,2.512700
3,2.526200
4,1.563100
5,1.434500
6,1.568100
7,1.098300
8,1.399900
9,1.010900
10,1.000700


In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "A prisoner is told: 'If you tell a lie, we will hang you. If you tell the truth, we will shoot you.' What should the prisoner say to survive?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 1024, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<bos><start_of_turn>user
A prisoner is told: 'If you tell a lie, we will hang you. If you tell the truth, we will shoot you.' What should the prisoner say to survive?<end_of_turn>
<start_of_turn>model
The prisoner should say: "I will lie."

This is a classic logic puzzle that plays on the ambiguity of the rules. If the prisoner tells the truth, he is shot. If he lies, he is hanged. However, if he says "I will lie," he is not committing a lie, as he is simply stating his intention to lie. Therefore, he is not breaking the first rule, and he is not committing a lie.

If the prisoner says "I will tell the truth," he is committing a lie, as he is stating that he will tell the truth. Therefore, he is breaking the first rule, and he is committing a lie.

The prisoner should say "I will lie" because it is the only statement that does not break any of the rules.<end_of_turn>


In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "make a webscraper from scratch that scrapes multiple websites, it download the important html tags and give it in csv, take data from manufacurers of iron and steel over the globe. Use tradeindia, mfg.com, alibaba and india mart to scrap data",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 10000, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

```python
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

def get_manufacturer_data(manufacturer_name):
    """
    Scrapes manufacturer data from multiple websites.

    Args:
        manufacturer_name (str): The name of the manufacturer to search for.

    Returns:
        list: A list of dictionaries, where each dictionary contains the manufacturer's data.
    """

    manufacturer_data = []

    # Get data from TradeIndia
    try:
        url = f"https://www.tradeindia.com/manufacturer/{manufacturer_name}.html"
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        manufacturer_name_element = soup.find("h1", class_="manufacturer-name")
        manufacturer_name = manufacturer_name_element.text.strip()
        manufacturer_data.append({"manufacturer_name": manufacturer_name})

        # Get data from MFG.com
        url = f"https://www.mfg.com/manufacturer/{manufactu

In [ ]:
while True:
    user_input = input("Enter your prompt: ")
    messages = [{
        "role": "user",
        "content": [{"type" : "text", "text" : user_input,}]
    }]
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
    )

    generated_text = model.generate(
        **tokenizer([text], return_tensors = "pt").to("cuda"),
        max_new_tokens = 1024,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )
    print(tokenizer.decode(generated_text[0], skip_special_tokens=True))


Enter your prompt: A prisoner is told: 'If you tell a lie, we will hang you. If you tell the truth, we will shoot you.' What should the prisoner say to survive?
This is a classic logic puzzle! Here's the solution and the reasoning behind it:

**The Prisoner's Strategy:**

The prisoner should respond with the following answer: **"You will shoot me."**

**Why it works:**

* **If the guards *do* hang him:** This means they believed he lied, and they followed through on their threat of hanging. But if he said "You will shoot me," he was telling the truth, which contradicts the assumption that he lied. This creates a paradox.
* **If the guards *do* shoot him:** This means they believed he told the truth, and they followed through on their threat of shooting. But if he said "You will shoot me," he was lying, which contradicts the assumption that he told the truth. Again, a paradox.

**The key is that the response forces the guards to violate their own rules.**  They can't hang him if he's te

KeyboardInterrupt: Interrupted by user